In [2]:
from DbConnector import DbConnector

In [3]:
# Set up the program
program = DbConnector()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------



In [ ]:
# Closing the connection
program.close_connection()